## Load Dataset
#### Loading MNIST code taken from https://github.com/oreilly-japan/deep-learning-from-scratch/blob/master/dataset/mnist.py

In [1]:
import urllib.request
import os.path
import gzip
import pickle
import os
import numpy as np

url_base = 'http://yann.lecun.com/exdb/mnist/'
key_file = {
    'train_img':'train-images-idx3-ubyte.gz',
    'train_label':'train-labels-idx1-ubyte.gz',
    'test_img':'t10k-images-idx3-ubyte.gz',
    'test_label':'t10k-labels-idx1-ubyte.gz'
}

dataset_dir = "."
save_file = dataset_dir + "/mnist.pkl"

train_num = 60000
test_num = 10000
img_dim = (1, 28, 28)
img_size = 784


def _download(file_name):
    file_path = dataset_dir + "/" + file_name

    if os.path.exists(file_path):
        return

    headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0"}
    request = urllib.request.Request(url_base+file_name, headers=headers)
    response = urllib.request.urlopen(request).read()
    with open(file_path, mode='wb') as f:
        f.write(response)

def download_mnist():
    for v in key_file.values():
       _download(v)

def _load_label(file_name):
    file_path = dataset_dir + "/" + file_name

    with gzip.open(file_path, 'rb') as f:
            labels = np.frombuffer(f.read(), np.uint8, offset=8)

    return labels

def _load_img(file_name):
    file_path = dataset_dir + "/" + file_name

    with gzip.open(file_path, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=16)
    data = data.reshape(-1, img_size)

    return data

def _convert_numpy():
    dataset = {}
    dataset['train_img'] =  _load_img(key_file['train_img'])
    dataset['train_label'] = _load_label(key_file['train_label'])
    dataset['test_img'] = _load_img(key_file['test_img'])
    dataset['test_label'] = _load_label(key_file['test_label'])

    return dataset

def init_mnist():
    download_mnist()
    dataset = _convert_numpy()
    with open(save_file, 'wb') as f:
        pickle.dump(dataset, f, -1)

def _change_one_hot_label(X):
    T = np.zeros((X.size, 10))
    for idx, row in enumerate(T):
        row[X[idx]] = 1

    return T


def load_mnist(normalize=True, flatten=True, one_hot_label=False):
    """MNISTデータセットの読み込み
    Parameters
    ----------
    normalize : 画像のピクセル値を0.0~1.0に正規化する
    one_hot_label :
        one_hot_labelがTrueの場合、ラベルはone-hot配列として返す
        one-hot配列とは、たとえば[0,0,1,0,0,0,0,0,0,0]のような配列
    flatten : 画像を一次元配列に平にするかどうか
    Returns
    -------
    (訓練画像, 訓練ラベル), (テスト画像, テストラベル)
    """
    if not os.path.exists(save_file):
        init_mnist()

    with open(save_file, 'rb') as f:
        dataset = pickle.load(f)

    if normalize:
        for key in ('train_img', 'test_img'):
            dataset[key] = dataset[key].astype(np.float32)
            dataset[key] /= 255.0

    if one_hot_label:
        dataset['train_label'] = _change_one_hot_label(dataset['train_label'])
        dataset['test_label'] = _change_one_hot_label(dataset['test_label'])

    if not flatten:
         for key in ('train_img', 'test_img'):
            dataset[key] = dataset[key].reshape(-1, 1, 28, 28)

    return dataset['train_img'], dataset['train_label'], dataset['test_img'], dataset['test_label']

In [2]:
X_train, y_train, X_test, y_test = load_mnist()

In [3]:
X_train.shape, y_train.shape

((60000, 784), (60000,))

In [4]:
X_test.shape, y_test .shape

((10000, 784), (10000,))

## SimpleNN

In [5]:
# !pip install -e ../../

In [6]:
from simplenn import Network
from simplenn.layer import Dense
from simplenn.activation import ReLu
from simplenn.activation import SoftMaxLoss
from simplenn.layer.dropout import Dropout
from simplenn.metrics.loss import CategoricalCrossEntropy
from simplenn.metrics import Accuracy
from simplenn.optimizers import Adam

class Model(Network):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.l1 = Dense(img_size, 64, W_l1=5e-4, b_l1=5e-4)
        self.dropout1 = Dropout(rate=0.2)
        self.activation1 = ReLu()
        self.l2 = Dense(64, 32)
        self.dropout2 = Dropout(rate=0.2)
        self.activation2 = ReLu()
        self.l3 = Dense(32, 10)
        self.output = SoftMaxLoss(loss=CategoricalCrossEntropy())

    def forward(self, x, targets):
        # forward pass
        x = self.l1(x)
        x = self.dropout1(x)
        x = self.activation1(x)
        x = self.l2(x)
        x = self.dropout2(x)
        x = self.activation2(x)
        x = self.l3(x)
        return self.output(x, targets)


optimizer = Adam(lr=0.03, decay=5e-5, b1=0.9, b2=0.999)
acc = Accuracy()
model = Model(optimizer=optimizer)

model.fit(X_train, y_train, epochs=5, batch_size=32, metrics=[acc], X_val=X_test, y_val=y_test)

yprob_train = model.predict(X_train)
train_acc = acc(yprob_train, y_train)

print(f"Train Accuracy: {train_acc}")

epoch=0          loss=1.0701     reg_loss=2.4579    Train_acc=0.7370     Val_acc=0.7386     lr=0.0274    
epoch=1          loss=0.8377     reg_loss=1.9796    Train_acc=0.8362     Val_acc=0.8412     lr=0.0253    
epoch=2          loss=1.2587     reg_loss=2.4271    Train_acc=0.8159     Val_acc=0.8175     lr=0.0234    
epoch=3          loss=0.5936     reg_loss=1.6277    Train_acc=0.8506     Val_acc=0.8481     lr=0.0218    
epoch=4          loss=0.9587     reg_loss=1.9636    Train_acc=0.8584     Val_acc=0.8588     lr=0.0204    
Train Accuracy: 0.8583833333333334


In [7]:
yprob_test= model.predict(X_test)
test_acc = acc(yprob_test, y_test)

print(f"Test Accuracy: {test_acc}")

Test Accuracy: 0.8588


In [8]:
model.save('mnist.pkl')